In [68]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [69]:
# df = pd.read_table('Data_28_F29.txt', delimiter=',')
df = pd.read_csv('cotton_data_02.csv')

In [70]:
df.head()

,Unnamed: 0,ID,FIELD,YLD03,ELE,SLOPE,CURV,PRO,PLAN,EC_SH,...,BAND3,BAND4,VI02_518,VI02_526,VI02_619,VI02_713,VI02_721,VI02_830,VI02_907,VI02_923
0,0,3018,C29,1082,1080.06,0.0879,0.002694,-0.001178,0.001515,14.4,...,123,82,-0.194690,-0.211382,-0.210084,-0.081545,0.040462,0.359116,0.273810,0.026178
1,1,3019,C29,1202,1080.08,0.0575,0.001403,-0.000473,0.000930,14.3,...,126,85,-0.192982,-0.220077,-0.221757,-0.115702,0.053254,0.439560,0.345238,0.096774
2,2,3020,C29,1196,1080.10,0.0906,-0.004624,0.004098,-0.000527,13.9,...,127,83,-0.189427,-0.218391,-0.217021,-0.100000,0.052632,0.445652,0.345238,0.124324
3,3,3021,C29,1120,1080.13,0.1291,-0.001335,0.001011,-0.000324,14.1,...,127,84,-0.192982,-0.221374,-0.215190,-0.128000,0.052632,0.425532,0.345238,0.104167
4,4,3022,C29,1119,1080.18,0.1515,0.002538,-0.001229,0.001309,14.8,...,130,85,-0.192982,-0.218045,-0.219917,-0.128000,0.017341,0.387097,0.321637,0.094737


In [71]:
# columns = ['AVGYLD','ELE','SLOPE','CURV', 'PRO','PLAN','EC_SH','EC_DP','BAND1','BAND2','BAND3','BAND4',
#            'VI00_520','VI00_528','VI00_613','VI00_707','VI00_715','VI00_723','VI00_816','VI00_824','VI00_901','VI00_917',
#            'VI01_616','VI01_624','VI01_710','VI01_827','VI01_912','VI02_518','VI02_526','VI02_619',
#            'VI02_713','VI02_721','VI02_830','VI02_907','VI02_923','VI03_505','VI03_529','VI03_606',
#            'VI03_622','VI03_724','VI03_825','VI03_926']
# 'VI03_505','VI03_529','VI03_606', 'VI03_622','VI03_724','VI03_825','VI03_926'

In [72]:

df_use = df.loc[:, df.columns != 'ID']

df_use = df_use[df_use['FIELD'] == 'C29']
df_use = df_use.loc[:, df_use.columns != 'FIELD']

df_use.head()

,Unnamed: 0,YLD03,ELE,SLOPE,CURV,PRO,PLAN,EC_SH,EC_DP,BAND1,...,BAND3,BAND4,VI02_518,VI02_526,VI02_619,VI02_713,VI02_721,VI02_830,VI02_907,VI02_923
0,0,1082,1080.06,0.0879,0.002694,-0.001178,0.001515,14.4,38.700001,96,...,123,82,-0.194690,-0.211382,-0.210084,-0.081545,0.040462,0.359116,0.273810,0.026178
1,1,1202,1080.08,0.0575,0.001403,-0.000473,0.000930,14.3,40.900002,94,...,126,85,-0.192982,-0.220077,-0.221757,-0.115702,0.053254,0.439560,0.345238,0.096774
2,2,1196,1080.10,0.0906,-0.004624,0.004098,-0.000527,13.9,36.900002,92,...,127,83,-0.189427,-0.218391,-0.217021,-0.100000,0.052632,0.445652,0.345238,0.124324
3,3,1120,1080.13,0.1291,-0.001335,0.001011,-0.000324,14.1,37.099998,96,...,127,84,-0.192982,-0.221374,-0.215190,-0.128000,0.052632,0.425532,0.345238,0.104167
4,4,1119,1080.18,0.1515,0.002538,-0.001229,0.001309,14.8,36.099998,96,...,130,85,-0.192982,-0.218045,-0.219917,-0.128000,0.017341,0.387097,0.321637,0.094737


In [73]:
res_final = df[df['FIELD'] == 'C29']
res_final = res_final[['ID', 'FIELD']]
res_final.head()

,ID,FIELD
0,3018,C29
1,3019,C29
2,3020,C29
3,3021,C29
4,3022,C29


In [74]:
df_use.columns.values

array(['Unnamed: 0', 'YLD03', 'ELE', 'SLOPE', 'CURV', 'PRO', 'PLAN',
       'EC_SH', 'EC_DP', 'BAND1', 'BAND2', 'BAND3', 'BAND4', 'VI02_518',
       'VI02_526', 'VI02_619', 'VI02_713', 'VI02_721', 'VI02_830',
       'VI02_907', 'VI02_923'], dtype=object)

In [75]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
import scikitplot as skplt

def compute_score(clf, X, y, scoring='mean_squared_error'):
    xval = cross_val_score(clf, X, y, cv = 5, scoring=scoring)
    return np.mean(xval)

def plot_roc(x_test, y_test, model):
    predicted_probas = model.predict_proba(x_test)
    skplt.metrics.plot_roc(y_test, predicted_probas, figsize =(10,10))
    plt.show()
    
def yld_category(yld):
    if yld <= 603:
        return 1
    elif 603 < yld <= 766:
        return 2
    elif 766 < yld <= 882:
        return 3
    elif 882 < yld <= 956:
        return 4
    elif 956 < yld <= 1018:
        return 5
    elif 1018 < yld <= 1075:
        return 6
    elif 1075 < yld <= 1134:
        return 7
    elif 1134 < yld <= 1200:
        return 8
    elif 1200 < yld <= 1280:
        return 9
    elif yld > 1280:
        return 10

In [76]:
full = df_use
full['AVGYLD_Category'] = full['YLD03'].apply(lambda x: yld_category(x))

In [77]:
# sns.distplot(full['AVGYLD_Category'], kde=False, bins=10, hist_kws={'edgecolor':'white', 'alpha':1.0})

In [78]:
# sns.distplot(full['AVGYLD_Category'], kde=False, bins=10, hist_kws={"rwidth":1,'edgecolor':'white', 'alpha':1.0})

In [79]:
tmp = []

for i in range(1, 11):
    tmp.append(full[full['AVGYLD_Category'] == i])

In [80]:
X_train = []
X_test = []
y_train = []
y_test = []
X_cols = [col for col in full.columns if col not in ['YLD03', 'AVGYLD_Category']]
X = []
y = []


for i in range(10):
    X.append(tmp[i][X_cols])
    y.append(tmp[i]['YLD03'])
    X_train_part, X_test_part, y_train_part, y_test_part = train_test_split(tmp[i][X_cols], tmp[i]['YLD03'], test_size=.2,random_state=0)
    X_train.append(X_train_part)
    X_test.append(X_test_part)
    y_train.append(y_train_part)
    y_test.append(y_test_part)

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(full[X_cols], full['YLD03'], test_size=.2,random_state=0)


X.append(full[X_cols])
y.append(full['YLD03'])
X_train.append(X_train_full)
X_test.append(X_test_full)
y_train.append(y_train_full)
y_test.append(y_test_full)



## dt

In [81]:
warnings.filterwarnings('ignore')
dt_tune_run = True
dt_parameters = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    print(i)
    defaut = {'criterion': 'mse', 'max_depth': None,
                  'max_leaf_nodes': None, 'min_impurity_split': 0.005, 'min_weight_fraction_leaf': 0.0,
              'presort': False, 'splitter': 'random'}
    dt_parameters.append(defaut)
    if dt_tune_run:
        parameter_grid = {
    #          'max_features':range(0,1,1),
            'random_state':range(0,10,1),
            'min_samples_leaf':range(1,10,1),
            'min_samples_split': range(2, 20, 1)
        }
        forest = DecisionTreeRegressor(criterion='mse', max_depth=None,
                                        max_leaf_nodes=None,
                                        min_impurity_split=0.005, min_weight_fraction_leaf=0.0,
                                        presort=False, splitter='random')

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        DecisionTreeModel = grid_search
        parameters = grid_search.best_params_
        dt_parameters[i]['min_samples_leaf'] = grid_search.best_params_['min_samples_leaf']
        dt_parameters[i]['min_samples_split'] = grid_search.best_params_['min_samples_split']
        dt_parameters[i]['random_state'] = grid_search.best_params_['random_state']
        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

10
Best score: -18931.81865546896
Best parameters: {'min_samples_leaf': 1, 'min_samples_split': 19, 'random_state': 0}


In [82]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
dt_model = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    tmp_model = DecisionTreeRegressor(**dt_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    dt_model.append(tmp_model)

for i in range(10, 11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(dt_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: dt range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: dt range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: dt range - {} measure: rmse, score: {}'.format(i+1, tmp))

10
Model: dt range - 11, measure: mae, score: 109.6
Model: dt range - 11 measure: mse, score: 18931.82
Model: dt range - 11 measure: rmse, score: 137.59


In [ ]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def me(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return max(np.abs(y_true - y_pred))

In [ ]:

for i in range(10, 11):
    print(i)
    y_pred = dt_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: dt range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: dt range - {}, measure: me, score: {}'.format(i+1, me_res))

In [ ]:
y_pred_final = dt_model[10].predict(full[X_cols])
dt_pred_df = pd.DataFrame({'predicted_dt':y_pred_final})
dt_pred_df.head()
res_final_after_dt = res_final.join(dt_pred_df)
res_final_after_dt.head()

## svm

In [83]:
warnings.filterwarnings('ignore')
svm_tune_run = True
svm_parameters = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    print(i)
    defaut = {'cache_size': 200, 'coef0': 0.0,
                  'degree': 3, 'kernel': 'rbf', 'max_iter': -1,
              'shrinking': True, 'tol': 0.001, 'verbose': False}
    svm_parameters.append(defaut)
    if svm_tune_run:
        parameter_grid = {
             'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
             'gamma' : [0.001, 0.01, 0.1, 1]
         }

        forest = SVR(cache_size=200, coef0=0.0,
                  degree=3, kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        SVCModel = grid_search
        parameters = grid_search.best_params_
        svm_parameters[i]['C'] = grid_search.best_params_['C']
        svm_parameters[i]['gamma'] = grid_search.best_params_['gamma']

        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

10
Best score: -32418.924987645798
Best parameters: {'C': 100, 'gamma': 0.001}


In [84]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
svm_model = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    tmp_model = SVR(**svm_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    svm_model.append(tmp_model)

for i in range(10, 11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(svm_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: svm range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: svm range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: svm range - {} measure: rmse, score: {}'.format(i+1, tmp))

10
Model: svm range - 11, measure: mae, score: 140.71
Model: svm range - 11 measure: mse, score: 32418.92
Model: svm range - 11 measure: rmse, score: 180.05


In [ ]:

for i in range(10, 11):
    print(i)
    y_pred = svm_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: svm range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: svm range - {}, measure: me, score: {}'.format(i+1, me_res))

In [ ]:
y_pred_final = svm_model[10].predict(full[X_cols])
svm_pred_df = pd.DataFrame({'predicted_svm':y_pred_final})
svm_pred_df.head()
res_final_after_svm = res_final_after_dt.join(svm_pred_df)
res_final_after_svm.head()

## random forest

In [85]:
warnings.filterwarnings('ignore')
rf_tune_run = True
rf_parameters = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    print(i)
    defaut = {'criterion': 'mse', 'max_depth': None, 'max_leaf_nodes': None,
                      'min_impurity_split': 0.005, 'min_weight_fraction_leaf': 0.0}
    rf_parameters.append(defaut)
    if rf_tune_run:
        parameter_grid = {
    #          'max_features':range(0,1,1),
            'random_state':range(0,10,1),
            'min_samples_leaf':range(1,10,1),
            'min_samples_split': range(2, 20, 1)
        }

        forest = RandomForestRegressor(criterion='mse', max_depth=None,
                                    max_leaf_nodes=None,
                                    min_impurity_split=0.005, min_weight_fraction_leaf=0.0)

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        SVCModel = grid_search
        parameters = grid_search.best_params_
        rf_parameters[i]['random_state'] = grid_search.best_params_['random_state']
        rf_parameters[i]['min_samples_leaf'] = grid_search.best_params_['min_samples_leaf']
        rf_parameters[i]['min_samples_split'] = grid_search.best_params_['min_samples_split']

        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

10
Best score: -19162.5108003998
Best parameters: {'min_samples_leaf': 3, 'min_samples_split': 7, 'random_state': 6}


In [86]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
rf_model = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    tmp_model = RandomForestRegressor(**rf_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    rf_model.append(tmp_model)

for i in range(10, 11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(rf_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: rf range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: rf range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: rf range - {} measure: rmse, score: {}'.format(i+1, tmp))

10
Model: rf range - 11, measure: mae, score: 109.24
Model: rf range - 11 measure: mse, score: 19162.51
Model: rf range - 11 measure: rmse, score: 138.43


In [ ]:
for i in range(10, 11):
    print(i)
    y_pred = rf_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: rf range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: rf range - {}, measure: me, score: {}'.format(i+1, me_res))

In [ ]:
y_pred_final = rf_model[10].predict(full[X_cols])
rf_pred_df = pd.DataFrame({'predicted_rf':y_pred_final})
rf_pred_df.head()

In [ ]:
res_final_after_rf = res_final_after_svm.join(rf_pred_df)
res_final_after_rf.head()

## xgboost

In [87]:
from xgboost import XGBRegressor
warnings.filterwarnings('ignore')

xgb_tune_run = True
xgb_parameters = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    print(i)
    defaut = {'subsample': 0.8, 'colsample_bytree': 1.0, 'max_depth': 2, 'gamma': 0.3, 'min_child_weight': 4}
    xgb_parameters.append(defaut)
    if xgb_tune_run:
        parameter_grid = {
                    'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)], 
                    'subsample':[i/10.0 for i in range(6,11)],
                    'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]
        }

        forest = XGBRegressor(nthread=-1)

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        SVCModel = grid_search
        parameters = grid_search.best_params_
        xgb_parameters[i]['min_child_weight'] = grid_search.best_params_['min_child_weight']
        xgb_parameters[i]['gamma'] = grid_search.best_params_['gamma']
        xgb_parameters[i]['subsample'] = grid_search.best_params_['subsample']
        xgb_parameters[i]['colsample_bytree'] = grid_search.best_params_['colsample_bytree']
        xgb_parameters[i]['max_depth'] = grid_search.best_params_['max_depth']
        
        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

10
Best score: -16939.6473070051
Best parameters: {'colsample_bytree': 0.7, 'gamma': 0.3, 'max_depth': 4, 'min_child_weight': 5, 'subsample': 1.0}


In [88]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
xgb_model = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    tmp_model = XGBRegressor(**xgb_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    xgb_model.append(tmp_model)

for i in range(10, 11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(xgb_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: xbg range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: xbg range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: xbg range - {} measure: rmse, score: {}'.format(i+1, tmp))

10
Model: xbg range - 11, measure: mae, score: 101.73
Model: xbg range - 11 measure: mse, score: 16939.65
Model: xbg range - 11 measure: rmse, score: 130.15


In [ ]:
for i in range(10, 11):
    print(i)
    y_pred = xgb_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: xgb range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: xgb range - {}, measure: me, score: {}'.format(i+1, me_res))

In [ ]:
y_pred_final = xgb_model[10].predict(full[X_cols])
xgb_pred_df = pd.DataFrame({'predicted_xgb':y_pred_final})
xgb_pred_df.head()

In [ ]:
res_final_after_xgb = res_final_after_rf.join(xgb_pred_df)
res_final_after_xgb.head()

## LinearRegression

In [89]:
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

lr_tune_run = True
lr_parameters = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    print(i)
    lr_parameters.append({})
    if lr_tune_run:
        parameter_grid = {
             'copy_X': [True, False],
             'normalize': [True, False]
         }

        forest = LinearRegression()

        grid_search = GridSearchCV(forest,
                                   scoring='mean_squared_error',
                                   param_grid=parameter_grid,
                                   cv=5)

        grid_search.fit(X[i], y[i])
        parameters = grid_search.best_params_
        lr_parameters[i]['copy_X'] = grid_search.best_params_['copy_X']
        lr_parameters[i]['normalize'] = grid_search.best_params_['normalize']
        
        print('Best score: {}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))

10
Best score: -27374.059733393035
Best parameters: {'copy_X': True, 'normalize': False}


In [90]:
warnings.filterwarnings('ignore')
score_measure_list = ['neg_mean_absolute_error', 'neg_mean_squared_error']
lr_model = [0,0,0,0,0,0,0,0,0,0]

for i in range(10, 11):
    tmp_model = LinearRegression(**lr_parameters[i])
    tmp_model.fit(X_train[i], y_train[i])
    lr_model.append(tmp_model)

for i in range(10, 11):
    print(i)
    for score_measure in score_measure_list:
        res = compute_score(lr_model[i], X[i], y[i], scoring = score_measure)
        res = round(res, 2) * -1
        if score_measure == 'neg_mean_absolute_error':
            print('Model: lr range - {}, measure: mae, score: {}'.format(i+1, res))
        elif score_measure == 'neg_mean_squared_error':
            print('Model: lr range - {} measure: mse, score: {}'.format(i+1, res))
            tmp = round(res ** 0.5, 2)
            print('Model: lr range - {} measure: rmse, score: {}'.format(i+1, tmp))

10
Model: lr range - 11, measure: mae, score: 126.94
Model: lr range - 11 measure: mse, score: 27374.06
Model: lr range - 11 measure: rmse, score: 165.45


In [ ]:
for i in range(10, 11):
    print(i)
    y_pred = lr_model[i].predict(X_test[i])
    mape_res = round(mape(y_test[i], y_pred),2)
    print('Model: lr range - {}, measure: mape, score: {}'.format(i+1, mape_res))
    me_res = round(me(y_test[i], y_pred),2)
    print('Model: lr range - {}, measure: me, score: {}'.format(i+1, me_res))

In [ ]:
y_pred_final = lr_model[10].predict(full[X_cols])
lr_pred_df = pd.DataFrame({'predicted_lr':y_pred_final})
lr_pred_df.head()

In [ ]:
res_final_after_lr = res_final_after_xgb.join(lr_pred_df)
res_final_after_lr.head()

In [ ]:
res_final_after_lr.to_csv('res_final_pred3.csv')